In [6]:
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets as widgets
import matplotlib.pyplot as plt
import plotly.express as pl
import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# from _future_ import print_function

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import folium
#import seaborn as sns


app = dash.Dash(__name__)

import plotly.graph_objects as go
%matplotlib inline

## data.world has a Creative Commons Open License
#  Dataset is 37,638,180 bytes (101.6 MB on disk)

In [5]:
#! conda install -c conda-forge folium --yes



Solving environment: done

## Package Plan ##

  environment location: /Users/lawrenceneely/opt/anaconda3/envs/DataViz

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       h033912b_0         138 KB  conda-forge
    certifi-2020.12.5          |   py37hf985489_1         143 KB  conda-forge
    folium-0.12.0              |     pyhd8ed1ab_1          64 KB  conda-forge
    openssl-1.1.1k             |       h0d85af4_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  folium             conda-forge/noarch::folium-0.12.0-pyhd8ed1ab_1

The following packages will be UPDATED:

  ca-certificates                      2020.6.20-hecda079_0 --> 2020.12.5-h033912b_0
  certifi  

In [7]:
# Dataset is from Covid-19 Data repository by the CSSE at Johns Hopkins University
# URL to retrieve it https://github.com/CSSEGISandData/COVID-19
cases_state = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_state.csv')
country_covid = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
covid_death = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',)
covid_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
covid_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
us_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/05-03-2021.csv')
us_cfdeath = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
us_states = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')

In [8]:
#data cleaning - rename province/state , country/region
covid_death = covid_death.rename(columns = {'Province/State': 'State', 'Country/Region': 'Country'})
covid_confirmed = covid_confirmed.rename(columns = {'Province/State': 'State', 'Country/Region': 'Country'})
covid_recovered = covid_recovered.rename(columns = {'Province/State': 'State', 'Country/Region': 'Country'})
country_covid = country_covid.rename(columns = {'Country_Region': 'country'})
us_confirmed = us_confirmed.rename(columns = {'Province_State': 'state', 'Country_Region': 'country'})

In [9]:
cases_state

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Australian Capital Territory,Australia,2021-05-13 19:31:14,-35.4735,149.0124,124,3,121.0,0.0,NaN,28.965195,NaN,NaN,2.419355,3601.0,AUS,NaN,NaN
1,New South Wales,Australia,2021-05-13 19:31:14,-33.8688,151.2093,5551,54,NaN,5497.0,NaN,68.378911,NaN,NaN,0.972798,3602.0,AUS,NaN,NaN
2,Northern Territory,Australia,2021-05-13 19:31:14,-12.4634,130.8456,167,0,163.0,4.0,NaN,67.996743,NaN,NaN,0.000000,3603.0,AUS,NaN,NaN
3,Queensland,Australia,2021-05-13 19:31:14,-27.4698,153.0251,1585,7,1560.0,18.0,NaN,30.984264,NaN,NaN,0.441640,3604.0,AUS,NaN,NaN
4,South Australia,Australia,2021-05-13 19:31:14,-34.9285,138.6007,741,4,722.0,15.0,NaN,42.186166,NaN,NaN,0.539811,3605.0,AUS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,"Saint Helena, Ascension and Tristan da Cunha",United Kingdom,2021-05-13 19:31:14,-7.9467,-14.3559,4,0,4.0,0.0,NaN,70.658894,NaN,NaN,0.000000,654.0,SHN,NaN,NaN
589,Scotland,United Kingdom,2021-05-13 19:31:14,56.4907,-4.2026,228693,7662,NaN,221031.0,NaN,4185.986492,NaN,NaN,3.350343,82603.0,GBR,NaN,NaN
590,Turks and Caicos Islands,United Kingdom,2021-05-13 19:31:14,21.6940,-71.7979,2404,17,2365.0,22.0,NaN,6208.998399,NaN,NaN,0.707155,796.0,TCA,NaN,NaN
591,Unknown,United Kingdom,2021-05-13 19:31:14,NaN,NaN,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,82605.0,GBR,NaN,NaN


In [10]:
covid_death

,State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,2642,2648,2654,2664,2673,2683,2686,2698,2710,2713
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2399,2402,2403,2406,2408,2411,2412,2416,2420,2423
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,3280,3289,3299,3307,3315,3321,3328,3335,3343,3350
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,127,127,127,127,127,127,127,127,127,127
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,603,609,618,622,628,630,633,636,639,645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,35,35,35,35,35,35,35,35,35,35
271,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,3283,3296,3317,3326,3338,3351,3358,3378,3393,3401
272,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1239,1240,1260,1265,1269,1270,1271,1275,1276,1277
273,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,1253,1254,1255,1256,1256,1257,1257,1257,1258,1259


In [11]:
country_covid

,country,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2021-05-13 19:20:54,33.939110,67.709953,63045.0,2721.0,54619.0,5705.0,161.951417,NaN,NaN,4.315965,4,AFG
1,Albania,2021-05-13 19:20:54,41.153300,20.168300,131890.0,2426.0,121122.0,8342.0,4583.014803,NaN,NaN,1.839412,8,ALB
2,Algeria,2021-05-13 19:20:54,28.033900,1.659600,124889.0,3355.0,87003.0,34531.0,284.802804,NaN,NaN,2.686386,12,DZA
3,Andorra,2021-05-13 19:20:54,42.506300,1.521800,13470.0,127.0,13104.0,239.0,17433.508057,NaN,NaN,0.942836,20,AND
4,Angola,2021-05-13 19:20:54,-11.202700,17.873900,29405.0,645.0,25187.0,3573.0,89.468631,NaN,NaN,2.193505,24,AGO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,2021-05-13 19:20:54,14.058324,108.277199,3710.0,35.0,2657.0,1018.0,3.811438,NaN,NaN,0.943396,704,VNM
188,West Bank and Gaza,2021-05-13 19:20:54,31.952200,35.233200,303270.0,3401.0,288891.0,10978.0,5944.820026,NaN,NaN,1.121443,275,PSE
189,Yemen,2021-05-13 19:20:54,15.552727,48.516388,6498.0,1277.0,3003.0,2218.0,21.786384,NaN,NaN,19.652201,887,YEM
190,Zambia,2021-05-13 19:20:54,-13.133897,27.849332,92211.0,1259.0,90574.0,378.0,501.584099,NaN,NaN,1.365347,894,ZMB


In [12]:
date = covid_confirmed.columns[4:]
tot_confirmed = covid_confirmed.melt(id_vars = ['State','Country','Lat','Long'],value_vars = date, var_name = 'date', 
                                    value_name = 'Confirmed')
tot_confirmed.tail()

,State,Country,Lat,Long,date,Confirmed
131170,NaN,Vietnam,14.058324,108.277199,5/12/21,3658
131171,NaN,West Bank and Gaza,31.952200,35.233200,5/12/21,303270
131172,NaN,Yemen,15.552727,48.516388,5/12/21,6498
131173,NaN,Zambia,-13.133897,27.849332,5/12/21,92211
131174,NaN,Zimbabwe,-19.015438,29.154857,5/12/21,38466


In [13]:
date2 = covid_death.columns[4:]
tot_death = covid_death.melt(id_vars = ['State','Country','Lat','Long'],value_vars = date, var_name = 'date', 
                                    value_name = 'Death')
tot_death.tail()

,State,Country,Lat,Long,date,Death
131170,NaN,Vietnam,14.058324,108.277199,5/12/21,35
131171,NaN,West Bank and Gaza,31.952200,35.233200,5/12/21,3401
131172,NaN,Yemen,15.552727,48.516388,5/12/21,1277
131173,NaN,Zambia,-13.133897,27.849332,5/12/21,1259
131174,NaN,Zimbabwe,-19.015438,29.154857,5/12/21,1579


In [14]:
covid_recovered

,State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,53391,53694,53750,53961,54019,54040,54222,54382,54503,54534
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,111792,112626,113468,114362,115253,116126,117089,118041,119061,120072
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,85534,85693,85869,86007,86149,86280,86420,86554,86703,86857
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,12793,12818,12884,12900,12936,12962,12995,13021,13070,13104
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,24190,24190,24492,24503,24698,24713,24717,24772,25145,25187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,2560,2560,2560,2560,2560,2602,2602,2618,2618,2636
256,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,276171,277673,279652,281586,283570,285575,286385,287206,287932,288891
257,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,2910,2948,2966,2971,2978,2983,2992,3001,3003,3003
258,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,90130,90166,90198,90264,90316,90363,90428,90501,90539,90574


In [15]:
date3 = covid_recovered.columns[4:]
tot_recovered = covid_recovered.melt(id_vars = ['State','Country','Lat','Long'],value_vars = date, var_name = 'date', 
                                    value_name = 'Recovered')
tot_recovered.tail()

,State,Country,Lat,Long,date,Recovered
124015,NaN,Vietnam,14.058324,108.277199,5/12/21,2636
124016,NaN,West Bank and Gaza,31.952200,35.233200,5/12/21,288891
124017,NaN,Yemen,15.552727,48.516388,5/12/21,3003
124018,NaN,Zambia,-13.133897,27.849332,5/12/21,90574
124019,NaN,Zimbabwe,-19.015438,29.154857,5/12/21,36277


In [16]:
covid = tot_confirmed.merge(right = tot_death, how = 'left', on = ['State','Country','date','Lat','Long'])
covid.tail()

,State,Country,Lat,Long,date,Confirmed,Death
131170,NaN,Vietnam,14.058324,108.277199,5/12/21,3658,35
131171,NaN,West Bank and Gaza,31.952200,35.233200,5/12/21,303270,3401
131172,NaN,Yemen,15.552727,48.516388,5/12/21,6498,1277
131173,NaN,Zambia,-13.133897,27.849332,5/12/21,92211,1259
131174,NaN,Zimbabwe,-19.015438,29.154857,5/12/21,38466,1579


In [17]:
covid = covid.merge(right = tot_recovered, how = 'left', on = ['State','Country','date','Lat','Long'])
covid.tail()

,State,Country,Lat,Long,date,Confirmed,Death,Recovered
131170,NaN,Vietnam,14.058324,108.277199,5/12/21,3658,35,2636.0
131171,NaN,West Bank and Gaza,31.952200,35.233200,5/12/21,303270,3401,288891.0
131172,NaN,Yemen,15.552727,48.516388,5/12/21,6498,1277,3003.0
131173,NaN,Zambia,-13.133897,27.849332,5/12/21,92211,1259,90574.0
131174,NaN,Zimbabwe,-19.015438,29.154857,5/12/21,38466,1579,36277.0


In [18]:
covid['date'] = pd.to_datetime(covid['date'])

In [19]:
covid.tail()

,State,Country,Lat,Long,date,Confirmed,Death,Recovered
131170,NaN,Vietnam,14.058324,108.277199,2021-05-12,3658,35,2636.0
131171,NaN,West Bank and Gaza,31.952200,35.233200,2021-05-12,303270,3401,288891.0
131172,NaN,Yemen,15.552727,48.516388,2021-05-12,6498,1277,3003.0
131173,NaN,Zambia,-13.133897,27.849332,2021-05-12,92211,1259,90574.0
131174,NaN,Zimbabwe,-19.015438,29.154857,2021-05-12,38466,1579,36277.0


In [20]:
covid.isna().sum()

State        90153
Country          0
Lat            954
Long           954
date             0
Confirmed        0
Death            0
Recovered    10017
dtype: int64

In [21]:
covid['Recovered'] = covid['Recovered'].fillna(0)

In [22]:
covid.isna().sum()

State        90153
Country          0
Lat            954
Long           954
date             0
Confirmed        0
Death            0
Recovered        0
dtype: int64

In [23]:
covid['Active'] = covid['Confirmed'] - covid['Death'] - covid['Recovered']

In [24]:
covid.head()

,State,Country,Lat,Long,date,Confirmed,Death,Recovered,Active
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,0.0


In [25]:
covid['date'].tail()

131170   2021-05-12
131171   2021-05-12
131172   2021-05-12
131173   2021-05-12
131174   2021-05-12
Name: date, dtype: datetime64[ns]

In [26]:
covid_confirmed.head()

,State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,60300,60563,60797,61162,61455,61755,61842,62063,62403,62718
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,131276,131327,131419,131510,131577,131666,131723,131753,131803,131845
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,122717,122999,123272,123473,123692,123900,124104,124288,124483,124682
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13295,13316,13340,13363,13390,13406,13423,13429,13447,13470
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,27133,27284,27529,27921,28201,28477,28740,28875,29146,29405


In [52]:
country_covid

,country,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2021-05-13 19:20:54,33.939110,67.709953,63045.0,2721.0,54619.0,5705.0,161.951417,NaN,NaN,4.315965,4,AFG
1,Albania,2021-05-13 19:20:54,41.153300,20.168300,131890.0,2426.0,121122.0,8342.0,4583.014803,NaN,NaN,1.839412,8,ALB
2,Algeria,2021-05-13 19:20:54,28.033900,1.659600,124889.0,3355.0,87003.0,34531.0,284.802804,NaN,NaN,2.686386,12,DZA
3,Andorra,2021-05-13 19:20:54,42.506300,1.521800,13470.0,127.0,13104.0,239.0,17433.508057,NaN,NaN,0.942836,20,AND
4,Angola,2021-05-13 19:20:54,-11.202700,17.873900,29405.0,645.0,25187.0,3573.0,89.468631,NaN,NaN,2.193505,24,AGO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,2021-05-13 19:20:54,14.058324,108.277199,3710.0,35.0,2657.0,1018.0,3.811438,NaN,NaN,0.943396,704,VNM
188,West Bank and Gaza,2021-05-13 19:20:54,31.952200,35.233200,303270.0,3401.0,288891.0,10978.0,5944.820026,NaN,NaN,1.121443,275,PSE
189,Yemen,2021-05-13 19:20:54,15.552727,48.516388,6498.0,1277.0,3003.0,2218.0,21.786384,NaN,NaN,19.652201,887,YEM
190,Zambia,2021-05-13 19:20:54,-13.133897,27.849332,92211.0,1259.0,90574.0,378.0,501.584099,NaN,NaN,1.365347,894,ZMB


In [55]:
country_covid
Clean = country_covid.dropna(subset=['Lat','Long_'])




In [56]:
Clean

,country,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2021-05-13 19:20:54,33.939110,67.709953,63045.0,2721.0,54619.0,5705.0,161.951417,NaN,NaN,4.315965,4,AFG
1,Albania,2021-05-13 19:20:54,41.153300,20.168300,131890.0,2426.0,121122.0,8342.0,4583.014803,NaN,NaN,1.839412,8,ALB
2,Algeria,2021-05-13 19:20:54,28.033900,1.659600,124889.0,3355.0,87003.0,34531.0,284.802804,NaN,NaN,2.686386,12,DZA
3,Andorra,2021-05-13 19:20:54,42.506300,1.521800,13470.0,127.0,13104.0,239.0,17433.508057,NaN,NaN,0.942836,20,AND
4,Angola,2021-05-13 19:20:54,-11.202700,17.873900,29405.0,645.0,25187.0,3573.0,89.468631,NaN,NaN,2.193505,24,AGO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,2021-05-13 19:20:54,14.058324,108.277199,3710.0,35.0,2657.0,1018.0,3.811438,NaN,NaN,0.943396,704,VNM
188,West Bank and Gaza,2021-05-13 19:20:54,31.952200,35.233200,303270.0,3401.0,288891.0,10978.0,5944.820026,NaN,NaN,1.121443,275,PSE
189,Yemen,2021-05-13 19:20:54,15.552727,48.516388,6498.0,1277.0,3003.0,2218.0,21.786384,NaN,NaN,19.652201,887,YEM
190,Zambia,2021-05-13 19:20:54,-13.133897,27.849332,92211.0,1259.0,90574.0,378.0,501.584099,NaN,NaN,1.365347,894,ZMB


In [28]:
covid_confirmedtot = int(country_covid['Confirmed'].sum())
covid_deathtot = int(country_covid['Deaths'].sum())
covid_recoveredtot = int(country_covid['Recovered'].sum())
cases_statetot = (country_covid['Confirmed'].sum)

In [29]:
covid_confirmedtot

160658300

In [30]:
cases_statetot

<bound method Series.sum of 0       63045.0
1      131890.0
2      124889.0
3       13470.0
4       29405.0
         ...   
187      3710.0
188    303270.0
189      6498.0
190     92211.0
191     38466.0
Name: Confirmed, Length: 192, dtype: float64>

In [88]:
# displaying the total stats

display(HTML("<div style = 'background-color:black; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;;display:inline-blockborder: 1px solid gray; vertical-align: middle; border-radius: 2px; background:blue '> Confirmed: "  + str(covid_confirmedtot) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;display:inline-blockborder: 1px solid gray; vertical-align: middle; border-radius: 2px; background:white'> Deaths: " + str(covid_deathtot) + "</span>"+
             "<span style='color: green; font-size:30px; margin-left:20px;display:inline-blockborder: 1px solid gray; vertical-align: middle; border-radius: 2px; background:orange'> Recovered: " + str(covid_recoveredtot) + "</span>"+
             "</div>")
       )


In [32]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_covid.sort_values('Confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [33]:
sort_countries = country_covid.sort_values('Deaths', ascending=False)

In [34]:
sort_countries

,country,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
178,US,2021-05-13 19:20:54,40.000000,-100.000000,32840007.0,584249.0,NaN,NaN,9967.638176,NaN,NaN,1.779077,840,USA
23,Brazil,2021-05-13 19:20:54,-14.235000,-51.925300,15359397.0,428034.0,13627986.0,1303377.0,7225.931363,NaN,NaN,2.786789,76,BRA
79,India,2021-05-13 19:20:54,20.593684,78.962880,23703665.0,258317.0,19734823.0,3710525.0,1717.651426,NaN,NaN,1.089777,356,IND
114,Mexico,2021-05-13 19:20:54,23.634500,-102.552800,2371483.0,219590.0,1893236.0,258657.0,1855.732513,NaN,NaN,9.259607,484,MEX
182,United Kingdom,2021-05-13 19:20:54,55.000000,-3.000000,4460404.0,127912.0,15201.0,4317291.0,6570.432397,NaN,NaN,2.867722,826,GBR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Samoa,2021-05-13 19:20:54,-13.759000,-172.104600,3.0,0.0,2.0,1.0,1.529598,NaN,NaN,0.000000,882,WSM
111,Marshall Islands,2021-05-13 19:20:54,7.131500,171.184500,4.0,0.0,4.0,0.0,6.847791,NaN,NaN,0.000000,584,MHL
144,Saint Kitts and Nevis,2021-05-13 19:20:54,17.357822,-62.782998,45.0,0.0,44.0,1.0,84.599188,NaN,NaN,0.000000,659,KNA
115,Micronesia,2021-05-13 19:20:54,7.425600,150.550800,1.0,0.0,1.0,0.0,0.878619,NaN,NaN,0.000000,583,FSM


In [35]:

def bubble_chart(n):
    
    fig = pl.scatter(sort_countries.head(), x="country", y="Deaths", size="Deaths", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Countries with the most deaths",
    xaxis_title="Countries",
    yaxis_title="Reported Deaths",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

In [62]:
Clean

,country,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2021-05-13 19:20:54,33.939110,67.709953,63045.0,2721.0,54619.0,5705.0,161.951417,NaN,NaN,4.315965,4,AFG
1,Albania,2021-05-13 19:20:54,41.153300,20.168300,131890.0,2426.0,121122.0,8342.0,4583.014803,NaN,NaN,1.839412,8,ALB
2,Algeria,2021-05-13 19:20:54,28.033900,1.659600,124889.0,3355.0,87003.0,34531.0,284.802804,NaN,NaN,2.686386,12,DZA
3,Andorra,2021-05-13 19:20:54,42.506300,1.521800,13470.0,127.0,13104.0,239.0,17433.508057,NaN,NaN,0.942836,20,AND
4,Angola,2021-05-13 19:20:54,-11.202700,17.873900,29405.0,645.0,25187.0,3573.0,89.468631,NaN,NaN,2.193505,24,AGO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,2021-05-13 19:20:54,14.058324,108.277199,3710.0,35.0,2657.0,1018.0,3.811438,NaN,NaN,0.943396,704,VNM
188,West Bank and Gaza,2021-05-13 19:20:54,31.952200,35.233200,303270.0,3401.0,288891.0,10978.0,5944.820026,NaN,NaN,1.121443,275,PSE
189,Yemen,2021-05-13 19:20:54,15.552727,48.516388,6498.0,1277.0,3003.0,2218.0,21.786384,NaN,NaN,19.652201,887,YEM
190,Zambia,2021-05-13 19:20:54,-13.133897,27.849332,92211.0,1259.0,90574.0,378.0,501.584099,NaN,NaN,1.365347,894,ZMB


In [74]:

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 8, min_zoom = 2)


for i in range(0,len(Clean)):
    folium.Circle(
        location=[Clean.iloc[i]['Lat'], Clean.iloc[i]['Long_']],
        fill=True,
       radius=(int((np.log(Clean.iloc[i]['Confirmed']+1.00001)))+0.2)*50000,
        color='orange',
        fill_color='lightgreen',
        tooltip = "<div style='margin: 0; background-color: white; color: gray;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+Clean.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: gray;'>"+
                    "<ul style='color: gray;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(Clean.iloc[i]['Confirmed'])+"</li>"+
                        "<li>Deaths:   "+str(Clean.iloc[i]['Deaths'])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(Clean.iloc[i]['Deaths']/(Clean.iloc[i]['Confirmed']+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map